Importation des libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews_all, reviews, app

Extraction des applications sur la plateforme Google Play Store française

In [1]:
url='https://play.google.com/store/search?q=health&c=apps&hl=fr'
req=requests.get(url)

soup = BeautifulSoup(req.content, 'html.parser')

cards= soup.find_all("div",class_="VfPpkd-EScbFb-JIbuQc TAQqTe")

On stocke les données des applications (identifiant de l'application, les descriptions, les liens, etc...)

In [2]:
Links = []
id = []
i = 0
for card in cards:
    app_name= card.find("span",class_="DdYX5").text
    company = card.find("span",class_="wMUdtb").text
    link = card.find("a", class_ ="Si6A0c Gy4nib")['href']
    #print(link+ "&hl=fr")
    #id.append(link[23:]+ "&hl=fr")
    id.append(link[23:])

    Links.append("https://play.google.com"+ link)

    #print("App: " +app_name)
    #print("Company: " + company)
    #print("Link : " + link)
    #print("id : " + id[i])
    #print(Links[i])
    i += 1


App: Samsung Health
Company: Samsung Electronics Co.,  Ltd.
Link : /store/apps/details?id=com.sec.android.app.shealth
id : com.sec.android.app.shealth
https://play.google.com/store/apps/details?id=com.sec.android.app.shealth
App: Google Fit: Activity Tracking
Company: Google LLC
Link : /store/apps/details?id=com.google.android.apps.fitness
id : com.google.android.apps.fitness
https://play.google.com/store/apps/details?id=com.google.android.apps.fitness
App: Health Connect (Beta)
Company: Google LLC
Link : /store/apps/details?id=com.google.android.apps.healthdata
id : com.google.android.apps.healthdata
https://play.google.com/store/apps/details?id=com.google.android.apps.healthdata
App: Health App
Company: Faraaz
Link : /store/apps/details?id=com.gmail.faraazvora010.healthapp
id : com.gmail.faraazvora010.healthapp
https://play.google.com/store/apps/details?id=com.gmail.faraazvora010.healthapp
App: Step Tracker - Pedometer
Company: Leap Fitness Group
Link : /store/apps/details?id=steptra

On teste sur la première application (Samsung Health)

In [ ]:
single_id = id[0]

On utilise Google Play Scraper pour extraire les commentaires

In [7]:
fr_reviews = reviews_all(
    single_id,
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

Affichage et stockage des données extraites

In [9]:
df_busu = pd.DataFrame(np.array(fr_reviews),columns=['review'])

df_busu = df_busu.join(pd.DataFrame(df_busu.pop('review').tolist()))

df_busu.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,39b2e192-bd25-438d-b59f-690e80127a92,Deebo,https://play-lh.googleusercontent.com/a-/ACB-R...,"Nicely designed, has a good selection of worko...",4,121,6.23.3.003,2023-03-05 21:11:25,"Hi, Deebo! We appreciate that you gave the app...",2023-03-06 18:27:23
1,5693c196-0361-47f7-ac35-4a5362154960,D. O'Neil,https://play-lh.googleusercontent.com/a/AGNmyx...,Good but buggy. There also appears to be no wa...,2,8,6.23.0.063,2023-03-07 02:11:22,"Hi, D. O'Neil! We are sorry to hear about your...",2023-03-07 19:14:35
2,18acbb16-1929-458e-a6b7-3121c6f2dfc3,Michael Gorman,https://play-lh.googleusercontent.com/a/AGNmyx...,Mediocre at best. Features frequently don't wo...,2,47,6.23.0.063,2023-02-10 03:10:28,"Hi, Michael Gorman! We are sorry to hear about...",2023-02-12 13:54:33
3,ce03f8f8-6203-44ac-a30a-4235c81f6540,Jacob Burgo,https://play-lh.googleusercontent.com/a-/ACB-R...,ECG is about as reliable as picking a piece of...,2,54,6.23.0.063,2023-02-03 20:36:39,"Hi, Jacob Burgo! We are sorry to hear about yo...",2023-02-05 16:03:02
4,76ed9820-b82b-464e-aac7-bd958f52c7da,Doris Allen,https://play-lh.googleusercontent.com/a-/ACB-R...,I have this synced with another app and used t...,4,31,6.23.0.063,2023-02-19 06:25:26,"Hi, Doris Allen! We are sorry to hear about yo...",2023-02-26 13:13:48


On enregistre le DataFrame sous forme d'un fichier csv pour traiter ces données

In [36]:
df_busu['content'].to_csv("GooglePlayReviewsFR.csv", index = False, header = True, encoding='utf-8')

Extraction des données des applications

In [27]:
result = app(
    single_id,
    lang='fr', # defaults to 'en'
    country='fr' # defaults to 'us'
)

Extraction des descriptions des applications

In [33]:
data = {'description':[]}

for i in id:
    result = app(
        i,
        lang='fr', # defaults to 'en'
        country='fr' # defaults to 'us'
    )
    data["description"].append(result["description"])

df = pd.DataFrame(data)
df.head()

,description
0,Commencez à prendre des habitudes saines grâce...
1,Optez pour une vie plus saine et plus active a...
2,"Avec Santé Connect par Android, partagez facil..."
3,"Que vous cherchiez à être plus actif, à perdre..."
4,Le tracker de pas le plus <b>simple et précis<...


On enregiste ces données sur un fichier csv

In [ ]:
df.to_csv("GooglePlayDescFR.csv", index = False, header = True, encoding='utf-8')